In [1]:
from pathlib import Path

from IPython.display import display
from ipywidgets import widgets

import logging

logger = logging.getLogger()

from dashboard_funcs import (
    create_start_standalone_command,
    write_shell_file,
    extract_archive,
    s3_download_file,
    s3_list_contents,
    write_fews_desktop_shortcut,
    write_voila_desktop_shortcut,
    write_voila_shell_file
)

BUCKET_NAME = "ciroh-rti-hefs-data"
FEWS_BASE_DIR = Path(Path.home(), "fews")
# FEWS_INSTALL_DIR = Path(FEWS_BASE_DIR, "binaries")
FEWS_INSTALL_DIR = Path("/opt", "fews")

FEWS_BASE_DIR.mkdir(exist_ok=True)
# FEWS_INSTALL_DIR.mkdir(exist_ok=True)

In [2]:
sa_dropdown = widgets.Dropdown(
    options=s3_list_contents(prefix="standalone_configurations"),
    description='Standalone:',
    disabled=False,
)

In [1]:
def install_fews_standalone(b) -> None:
    """Download standalone configuration from S3 to the working directory."""
    # 1. Download
    print("Downloading standalone tar.gz file...")
    sa_parent_dir = Path(FEWS_BASE_DIR, Path(sa_dropdown.value).parent)
    sa_filepath = Path(FEWS_BASE_DIR, sa_dropdown.value)
    sa_parent_dir.mkdir(exist_ok=True, parents=True)
    s3_download_file(sa_dropdown.value, sa_filepath)
    print("Download complete.")
    print("Extracting standalone configuration...")
    # 2. Extract
    extract_archive(sa_filepath, sa_parent_dir)
    # 3. Create the bash command to run the standalone
    sa_dir_path = list(sa_parent_dir.glob("*"))[0]  # TODO: Hacky?
    bash_command_str = create_start_standalone_command(
        FEWS_INSTALL_DIR.as_posix(),
        sa_dir_path
    )
    # 4. Write the bash command to a sh file and copy to Desktop
    shell_script_filepath = Path(sa_dir_path, "start_fews_standalone.sh")
    write_shell_file(shell_script_filepath, bash_command_str)
    # 5. Copy in patch file.
    print("Downloading patch file...")
    s3_download_file(
        "fews-install/fews-NA-202102-125264-patch.jar",
        Path(sa_dir_path, "fews-NA-202102-125264-patch.jar")
    )
    # 6. Create FEWS desktop shortcut.
    print("Creating FEWS desktop shortcut...")
    Path(Path.home(), "Desktop").mkdir(exist_ok=True)
    desktop_shortcut_filepath = Path(Path.home(), "Desktop", f"{sa_dir_path.name}.desktop")
    write_fews_desktop_shortcut(desktop_shortcut_filepath, shell_script_filepath)
    # # 7. Create the voila script and shortcut.
    # voila_shell_script_filepath = Path(Path.home(), "start_voila.sh")
    # bash_command_str = "voila /home/jovyan/dashboard.ipynb"
    # # write_shell_file(voila_shell_script_filepath, bash_command_str)
    # write_voila_shell_file(voila_shell_script_filepath)
    # desktop_shortcut_filepath = Path(Path.home(), "Desktop", "dashboard.desktop")
    # write_voila_desktop_shortcut(desktop_shortcut_filepath, voila_shell_script_filepath)
    print("Installation complete.")
    return

In [ ]:
config_button = widgets.Button(description='Configure Standalone')
config_button.on_click(install_fews_standalone)

### Configure FEWS for your region of interest

In [ ]:
display(sa_dropdown)

In [ ]:
display(config_button)